# Import thư viện

### Nạp thư viện

In [1]:
!python -m pip install --upgrade pip

! pip -q install --upgrade pip
! pip -q install -r req.txt
! pip install -U accelerate
! pip install -U transformers

  Using cached pip-25.1.1-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-25.1.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl (308 kB)

   -------------------- ------------------- 1/2 [accelerate]
   -------------------- ------------------- 1/2 [accelerate]
   -------------------- ------------------- 1/2 [accelerate]
   -------------------- ------------------- 1/2 [accelerate]
   -------------------- ------------------- 1/2 [accelerate]
   -------------------- ------------------- 1/2 [accelerate]
   -------------------- ------------------- 1/2 [accelerate]
   -------------------- ------------------- 1/2 [accelerate]
   -------------------- ------------------- 1/2 [accelerate]
   -------------------- ------------------- 1/2 [accelera

### Import thư viện

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder

d:\makerthon\phoBERT-finetuning\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Tiền xử lí dữ liệu



### Nạp model

In [10]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base-v2", num_labels=93)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### lưu model (optional)

In [11]:
model.save_pretrained("model")
tokenizer.save_pretrained("tokenizer")

('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\vocab.txt',
 'tokenizer\\bpe.codes',
 'tokenizer\\added_tokens.json')

### Nạp dataset

In [12]:
# Tải dữ liệu từ file CSV
dataset = load_dataset("csv", data_files={"train": "dataset.csv", "test": "dataset.csv"})

# Lấy danh sách các nhãn gốc từ cột 'intent'
intent_labels = dataset["train"]["intent"]  # Cột 'intent' chứa các nhãn gốc

# Khởi tạo LabelEncoder để mã hóa nhãn thành chỉ số
label_encoder = LabelEncoder()

# Mã hóa các nhãn gốc thành các chỉ số
label_encoder.fit(intent_labels)  # Fit trên toàn bộ nhãn từ cột 'intent'

# In ra các nhãn gốc và các chỉ số đã mã hóa
print("Danh sách nhãn gốc:", label_encoder.classes_)  # In mảng nhãn gốc
print("Các nhãn đã mã hóa:", label_encoder.transform(intent_labels))  # In nhãn đã mã hóa thành chỉ số

# Mã hóa nhãn intent thành chỉ số và tokenize văn bản
def encode_labels_and_tokenize(examples):
    # Tokenize văn bản
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=13)
    
    # Mã hóa intent thành nhãn (labels) sử dụng LabelEncoder đã được fit
    tokenized["labels"] = label_encoder.transform(examples["intent"])  # Mã hóa nhãn thành chỉ số
    
    return tokenized

# Áp dụng mã hóa nhãn và tokenization cho cả train và test dataset
train_dataset = dataset["train"].map(encode_labels_and_tokenize, batched=True)
test_dataset = dataset["test"].map(encode_labels_and_tokenize, batched=True)

# Kiểm tra dữ liệu sau khi mã hóa nhãn và tokenization
  
print(train_dataset["labels"])  # Hiển thị mẫu đầu tiên trong train dataset

Danh sách nhãn gốc: ['AI & Internet' 'An ninh mạng' 'An sinh' 'Bóng đá' 'Bất động sản'
 'Chuyện của tôi' 'Chuyện nghề' 'Chính sách' 'Chính trị' 'Chợ online'
 'Chứng khoán' 'Công nghệ' 'Công sở' 'Cộng đồng' 'Cộng đồng xe'
 'Doanh nghiệp' 'Du học' 'Du lịch' 'Dự án' 'Gia dụng thông minh'
 'Gia đình' 'Giao thông' 'Giáo dục' 'Giáo dục - Nghề nghiệp' 'Giải trí'
 'Giới trẻ' 'Góc phụ huynh' 'Gương sáng' 'Hậu trường' 'Học tập Bác'
 'Hồ sơ vụ án' 'Khoa học' 'Khoa học & đời sống' 'Khuyến học' 'Khám phá'
 'Khỏe đẹp' 'Khởi nghiệp' 'Kinh doanh' 'Kinh nghiệm - Tư vấn'
 'Kiến thức giới tính' 'Kiều bào' 'Kỷ nguyên mới' 'Làm giàu'
 'Món ngon - Điểm đẹp' 'Môi trường' 'Mỹ thuật - Sân khấu'
 'Ngoại thần kinh - Cột sống' 'Nhà đất' 'Nhân lực mới' 'Nhịp sống đô thị'
 'Nóng trên mạng' 'Nội thất' 'Nội vụ' 'Pháp đình' 'Phân tích - Bình luận'
 'Pickleball' 'Quân sự' 'Sách hay' 'Sản phẩm & Cộng đồng' 'Sống khỏe'
 'Sống xanh' 'Sức khỏe' 'Tennis' 'Thượng lưu' 'Thế giới'
 'Thế giới tự nhiên' 'Thế giới đó đây' 'Thể th

# Finetuning

### Cấu hình tham số huấn luyện

In [14]:
training_args = TrainingArguments(
    output_dir="./results",
    logging_dir='./logs',
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=10  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)


### Finetuning

In [15]:
trainer.train()

d:\makerthon\phoBERT-finetuning\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,4.529900
20,4.515400
30,4.485500
40,4.485000
50,4.422200
60,4.383000
70,4.328600
80,4.301800
90,4.209000
100,4.213600


TrainOutput(global_step=224, training_loss=4.1793670398848395, metrics={'train_runtime': 205.8744, 'train_samples_per_second': 8.646, 'train_steps_per_second': 1.088, 'total_flos': 11901102253560.0, 'train_loss': 4.1793670398848395, 'epoch': 4.0})

In [16]:
# Lưu mô hình và tokenizer của PhoBERT
tokenizer.save_pretrained("./results/final_model")
model.save_pretrained("./results/final_model")

# Test output

In [18]:
tokenizer = AutoTokenizer.from_pretrained("./results/final_model")
model = AutoModelForSequenceClassification.from_pretrained("./results/final_model")

In [20]:
# ['clear_history' 'find_api' 'find_history' 'log_in' 'log_out']

import torch

# Ví dụ văn bản muốn dự đoán
text = "tôi muốn tìm hiểu về bất động sản"

# Tokenize văn bản
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Dự đoán với mô hình
with torch.no_grad():
    outputs = model(**inputs)

# Lấy kết quả dự đoán (logits)
logits = outputs.logits

# Lấy nhãn dự đoán (với argmax)
predicted_label = logits.argmax(dim=-1).item()

print(f"Predicted label: {predicted_label}")


Predicted label: 4
